# 1. Haar Cascade(face detect) + Dlib(eye landmark) + EAR algorithm

### Haar Cascade를 이용해 얼굴 영역을 더 빠르게 찾을 수 있으나 인식률이 떨어짐
### 특히 측면 얼굴은 인식하지 못함

In [13]:
import numpy as np
import dlib
import cv2
import time
#import pygame

#pygame.mixer.init()
#pygame.mixer.music.load('./audio/fire-truck.wav')

RIGHT_EYE = list(range(36, 42))
LEFT_EYE = list(range(42, 48))
EYES = list(range(36, 48))
frame_width = 640
frame_height = 480

title_name = 'Face Drowsiness Detection'
elapsed_time = 0

face_cascade_name = cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml'
face_cascade = cv2.CascadeClassifier()
if not face_cascade.load(cv2.samples.findFile(face_cascade_name)):
    print('Error loading face cascade')
    exit(0)

predictor_file = 'study/model/shape_predictor_68_face_landmarks.dat'
predictor = dlib.shape_predictor(predictor_file)

status = 'Awake'
number_closed = 0
min_EAR = 0.25
closed_limit = 7
show_frame = None
sign = None
color = None

def getEAR(points): #EAR 값을 구하는 함수
    A = np.linalg.norm(points[1] - points[5])
    B = np.linalg.norm(points[2] - points[4])
    C = np.linalg.norm(points[0] - points[3])
    return (A + B) / (2.0 * C)
    
def detectAndDisplay(image):
    global number_closed
    global color
    global show_frame
    global sign
    global elapsed_time
    start_time = time.time()
    #height,width = image.shape[:2]
    image = cv2.resize(image, (frame_width, frame_height))
    show_frame = image
    frame_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #인식률을 높이기 위해 noise를 없앰
    frame_gray = cv2.equalizeHist(frame_gray)
    faces = face_cascade.detectMultiScale(frame_gray) #haar cascade를 이용해 face detect
    
    for (x,y,w,h) in faces: #haar cascade가 찾은 좌표
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        rect = dlib.rectangle(int(x), int(y), int(x + w), int(y + h))
        points = np.matrix([[p.x, p.y] for p in predictor(frame_gray, rect).parts()])
        show_parts = points[EYES]
        right_eye_EAR = getEAR(points[RIGHT_EYE]) #EAR 값 구함
        left_eye_EAR = getEAR(points[LEFT_EYE])
        mean_eye_EAR = (right_eye_EAR + left_eye_EAR) / 2 

        right_eye_center = np.mean(points[RIGHT_EYE], axis = 0).astype("int")
        left_eye_center = np.mean(points[LEFT_EYE], axis = 0).astype("int")

        cv2.putText(image, "{:.2f}".format(right_eye_EAR), (right_eye_center[0,0], right_eye_center[0,1] + 20),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        cv2.putText(image, "{:.2f}".format(left_eye_EAR), (left_eye_center[0,0], left_eye_center[0,1] + 20),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        
        for (i, point) in enumerate(show_parts): #눈의 landmark 표시
            x = point[0,0]
            y = point[0,1]
            cv2.circle(image, (x, y), 1, (0, 255, 255), -1)
            
        if mean_eye_EAR > min_EAR: #눈을 떴다고 판단
            color = (0, 255, 0)
            status = 'Awake'
            number_closed = number_closed - 1
            if( number_closed<0 ):
                number_closed = 0
        else: #눈을 감았다고 판단
            color = (0, 0, 255)
            status = 'Sleep'
            number_closed = number_closed + 1
                     
        sign = status + ', Sleep count : ' + str(number_closed) + ' / ' + str(closed_limit)
        if( number_closed > closed_limit ):
            show_frame = frame_gray
            # play SOUND
            #if(pygame.mixer.music.get_busy()==False):
                #pygame.mixer.music.play()

    cv2.putText(show_frame, sign , (10,frame_height-20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    cv2.imshow(title_name, show_frame)
    frame_time = time.time() - start_time
    elapsed_time += frame_time
    print("Frame time {:.3f} seconds".format(frame_time))
    

vs = cv2.VideoCapture(0)
time.sleep(2.0)
if not vs.isOpened:
    print('Error opening video')
    exit(0)
while True:
    ret, frame = vs.read()
    if frame is None:
        print('No more frame')
        vs.release()
        break
    detectAndDisplay(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


vs.release()
cv2.destroyAllWindows()


Frame time 0.232 seconds
Frame time 0.199 seconds
Frame time 0.199 seconds
Frame time 0.219 seconds
Frame time 0.220 seconds
Frame time 0.230 seconds
Frame time 0.213 seconds
Frame time 0.186 seconds
Frame time 0.215 seconds
Frame time 0.227 seconds
Frame time 0.209 seconds
Frame time 0.185 seconds
Frame time 0.186 seconds
Frame time 0.196 seconds
Frame time 0.201 seconds
Frame time 0.200 seconds
Frame time 0.202 seconds
Frame time 0.223 seconds
Frame time 0.212 seconds
Frame time 0.182 seconds
Frame time 0.205 seconds
Frame time 0.194 seconds
Frame time 0.216 seconds
Frame time 0.216 seconds
Frame time 0.208 seconds
Frame time 0.218 seconds
Frame time 0.132 seconds
Frame time 0.126 seconds
Frame time 0.127 seconds
Frame time 0.136 seconds
Frame time 0.128 seconds
Frame time 0.142 seconds
Frame time 0.146 seconds
Frame time 0.138 seconds
Frame time 0.141 seconds
Frame time 0.148 seconds
Frame time 0.131 seconds
Frame time 0.148 seconds
Frame time 0.149 seconds
Frame time 0.152 seconds


---
# 2. Dlib(face landmark + eye landmark) + EAR algorithm

### 이전보다 측면의 눈을 잘 인식함

In [1]:
import numpy as np
import dlib
import cv2
import time
#import pygame

#pygame.mixer.init()
#pygame.mixer.music.load('./audio/fire-truck.wav')

RIGHT_EYE = list(range(36, 42))
LEFT_EYE = list(range(42, 48))
EYES = list(range(36, 48))
frame_width = 640
frame_height = 480

title_name = 'Face Drowsiness Detection'
elapsed_time = 0

# face_cascade_name = cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml'
# face_cascade = cv2.CascadeClassifier()
# if not face_cascade.load(cv2.samples.findFile(face_cascade_name)):
#     print('Error loading face cascade')
#     exit(0)

predictor_file = 'study/model/shape_predictor_68_face_landmarks.dat'
predictor = dlib.shape_predictor(predictor_file)

####
detector = dlib.get_frontal_face_detector()

status = 'Awake'
number_closed = 0
min_EAR = 0.25
closed_limit = 7
show_frame = None
sign = None
color = None

def getEAR(points): #EAR 값을 구하는 함수
    A = np.linalg.norm(points[1] - points[5])
    B = np.linalg.norm(points[2] - points[4])
    C = np.linalg.norm(points[0] - points[3])
    return (A + B) / (2.0 * C)
    
def detectAndDisplay(image):
    global number_closed
    global color
    global show_frame
    global sign
    global elapsed_time
    start_time = time.time()
    #height,width = image.shape[:2]
    image = cv2.resize(image, (frame_width, frame_height))
    show_frame = image
    frame_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #인식률을 높이기 위해 noise를 없앰
    frame_gray = cv2.equalizeHist(frame_gray)
    faces = detector(frame_gray, 1) #haar cascade를 이용해 face detect
    
    for (i, face) in enumerate(faces): #haar cascade가 찾은 좌표
#         cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
#         rect = dlib.rectangle(int(x), int(y), int(x + w), int(y + h))
#         points = np.matrix([[p.x, p.y] for p in predictor(frame_gray, rect).parts()])
#         show_parts = points[EYES]
        points = np.matrix([[p.x, p.y] for p in predictor(frame_gray, face).parts()])
        show_parts = points[EYES]
        right_eye_EAR = getEAR(points[RIGHT_EYE]) #EAR 값 구함
        left_eye_EAR = getEAR(points[LEFT_EYE])
        mean_eye_EAR = (right_eye_EAR + left_eye_EAR) / 2 

        right_eye_center = np.mean(points[RIGHT_EYE], axis = 0).astype("int")
        left_eye_center = np.mean(points[LEFT_EYE], axis = 0).astype("int")

        cv2.putText(image, "{:.2f}".format(right_eye_EAR), (right_eye_center[0,0], right_eye_center[0,1] + 20),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        cv2.putText(image, "{:.2f}".format(left_eye_EAR), (left_eye_center[0,0], left_eye_center[0,1] + 20),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        
        for (i, point) in enumerate(show_parts): #눈의 landmark 표시
            x = point[0,0]
            y = point[0,1]
            cv2.circle(image, (x, y), 1, (0, 255, 255), -1)
            
        if mean_eye_EAR > min_EAR: #눈을 떴다고 판단
            color = (0, 255, 0)
            status = 'Awake'
            number_closed = number_closed - 1
            if( number_closed<0 ):
                number_closed = 0
        else: #눈을 감았다고 판단
            color = (0, 0, 255)
            status = 'Sleep'
            number_closed = number_closed + 1
                     
        sign = status + ', Sleep count : ' + str(number_closed) + ' / ' + str(closed_limit)
        if( number_closed > closed_limit ):
            show_frame = frame_gray
            # play SOUND
            #if(pygame.mixer.music.get_busy()==False):
                #pygame.mixer.music.play()

    cv2.putText(show_frame, sign , (10,frame_height-20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    cv2.imshow(title_name, show_frame)
    frame_time = time.time() - start_time
    elapsed_time += frame_time
    print("Frame time {:.3f} seconds".format(frame_time))
    

vs = cv2.VideoCapture(0)
time.sleep(2.0)
if not vs.isOpened:
    print('Error opening video')
    exit(0)
while True:
    ret, frame = vs.read()
    if frame is None:
        print('No more frame')
        vs.release()
        break
    detectAndDisplay(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


vs.release()
cv2.destroyAllWindows()


Frame time 0.728 seconds
Frame time 0.615 seconds
Frame time 0.559 seconds
Frame time 0.563 seconds
Frame time 0.588 seconds
Frame time 0.614 seconds
Frame time 0.634 seconds
Frame time 0.544 seconds
Frame time 0.514 seconds
Frame time 0.415 seconds
Frame time 0.352 seconds
Frame time 0.327 seconds
Frame time 0.287 seconds
Frame time 0.321 seconds
Frame time 0.306 seconds
Frame time 0.306 seconds
Frame time 0.295 seconds
Frame time 0.302 seconds
Frame time 0.322 seconds
Frame time 0.322 seconds
Frame time 0.380 seconds
Frame time 0.364 seconds
Frame time 0.386 seconds
Frame time 0.320 seconds
Frame time 0.444 seconds
Frame time 0.608 seconds
Frame time 0.525 seconds
Frame time 0.554 seconds
Frame time 0.525 seconds
Frame time 0.479 seconds
Frame time 0.489 seconds
Frame time 0.521 seconds
Frame time 0.328 seconds
Frame time 0.300 seconds
Frame time 0.317 seconds
Frame time 0.290 seconds
Frame time 0.292 seconds
Frame time 0.289 seconds
Frame time 0.301 seconds
Frame time 0.318 seconds


---
# 3. Dlib(face landmark + eye landmark) + tensorflow

### open eyes, closed eyes 의 이미지(약 3000개)를 학습
### 1. model 생성

In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import pickle
import dlib
import time
#deep learning model for training
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

Datadirectory = "Train_Dataset/" #training dataset
Classes = ["closed_eye", "open_eye"]
for category in Classes:
    path = os.path.join(Datadirectory, category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
        backtorgb = cv2.cvtColor(img_array, cv2.COLOR_GRAY2RGB)
#         plt.imshow(img_array, cmap="gray")
#         plt.show()
        break
    break

img_size = 224
new_array = cv2.resize(backtorgb, (img_size, img_size))
# plt.imshow(new_array, cmap="gray")
# plt.show()

#reading all the images and converting them into an array for data and labels
training_Data = []
def create_training_Data():
    for category in Classes:
        path = os.path.join(Datadirectory, category)
        class_num = Classes.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
                backtorgb = cv2.cvtColor(img_array, cv2.COLOR_GRAY2RGB)
                new_array = cv2.resize(backtorgb, (img_size, img_size))
                training_Data.append([new_array,class_num])
            except Exception as e:
                pass

create_training_Data()

import random
random.shuffle(training_Data)

X = []
y = []
for features,label in training_Data:
    X.append(features)
    y.append(label)
X = np.array(X).reshape(-1, img_size, img_size, 3)

#normalize the data
X = X/255.0

Y = np.array(y)

pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

model = tf.keras.applications.mobilenet.MobileNet()

# Trnasfer learning
base_input = model.layers[0].input

base_output = model.layers[-4].output

Flat_layer = layers.Flatten()(base_output)
final_output = layers.Dense(1)(Flat_layer) #one node
final_output = layers.Activation('sigmoid')(final_output)

new_model = keras.Model(inputs = base_input, outputs = final_output)

#settings for binary classification
new_model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

new_model.fit(X,Y,epochs=1,validation_split=0.1)

new_model.save('my_model.h5')

### 2. model 이용

In [5]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import pickle
import dlib
import time
#deep learning model for training
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
new_model = tf.keras.models.load_model('my_model.h5')
frame_width = 640
frame_height = 480

predictor_path = "study/model/shape_predictor_68_face_landmarks.dat"

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

number_closed = 0

path = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot open webcam")
    
while True:
    ret, frame = cap.read()
    #eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     eyes = eye_cascade.detectMultiScale(gray, 1.1, 4)
#     for x,y,w,h in eyes:
#         roi_gray = gray[y:y+h, x:x+w]
#         roi_color = frame[y:y+h, x:x+w]
#         cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)
#         eyess = eye_cascade.detectMultiScale(roi_gray)
#         if len(eyess) == 0:
#             print("eyes are not detected")

#         else:
#             for (ex, ey, ew, eh) in eyess:
#                 eyes_roi = roi_color[ey:ey+eh,ex:ex+ew]
    dets = detector(gray)
    for k, d in enumerate(dets):
        shape = predictor(gray, d)
    vec = np.empty([68, 2], dtype = int)
    x1 = shape.part(36).x
    x2 = shape.part(39).x
    y1 = shape.part(36).y
    img_crop = frame[y1-(x2-x1)//2:y1+(x2-x1)//2, x1:x2]
    final_image = cv2.resize(img_crop, (224,224))
    final_image = np.expand_dims(final_image,axis=0)
    final_image=final_image/255.0
    Predictions = new_model.predict(final_image)
    #print(Predictions)0.00000001
    if(Predictions>0.00000001):
        status = "Open Eyes"
    else:
        status = "Closed Eyes"
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #print(faceCascade.empty())
    #faces = faceCascade.detectMultiScale(gray,1.1,4)
    
#     for(x,y,w,h) in faces:
#         cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 2)
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    if status == "Open Eyes": #눈을 떴다고 판단
        color = (0, 255, 0)
        number_closed = number_closed - 1
        if( number_closed<0 ):
            number_closed = 0
    else: #눈을 감았다고 판단
        color = (0, 0, 255)
        number_closed = number_closed + 1
                     
    sign = status + ', Sleep count : ' + str(number_closed) + ' / ' + str(7)
#     if( number_closed > 7 ):
#         show_frame = frame_gray
            # play SOUND
            #if(pygame.mixer.music.get_busy()==False):
                #pygame.mixer.music.play()

    cv2.putText(frame, sign , (10,frame_height-20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    #cv2.imshow(title_name, frame)
#     frame_time = time.time() - start_time
#     elapsed_time += frame_time
#     print("Frame time {:.3f} seconds".format(frame_time))
    
    cv2.putText(frame, status, (50,50), font, 3, (0,0,255), 2, cv2.LINE_4)
    cv2.imshow('Drowsiness Detection Tutorial', frame)
    
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 64ms/step


---
# 4. Haar cascde(face detect+eye detect) + Tensorflow
### - 계획서에 썼던 내용
### - Haar cascde 인식률이 좋지 않음
### - tensorflow model은 위와 같은 모델을 사용

In [ ]:
#real time video demo
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import time
#deep learning model for training
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
new_model = tf.keras.models.load_model('my_model.h5')

path = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot open webcam")
    
while True:
    ret, frame = cap.read()
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray, 1.1, 4)
    for x,y,w,h in eyes:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)
        eyess = eye_cascade.detectMultiScale(roi_gray)
        if len(eyess) == 0:
            print("eyes are not detected")

        else:
            for (ex, ey, ew, eh) in eyess:
                eyes_roi = roi_color[ey:ey+eh,ex:ex+ew]
    final_image = cv2.resize(eyes_roi, (224,224))
    final_image = np.expand_dims(final_image,axis=0)
    final_image=final_image/255.0
    Predictions = new_model.predict(final_image)
    if(Predictions>0.00000001):
        status = "Open Eyes"
    else:
        status = "Closed Eyes"
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    print(faceCascade.empty())
    faces = faceCascade.detectMultiScale(gray,1.1,4)
    
    for(x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 2)
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    cv2.putText(frame, status, (50,50), font, 3, (0,0,255), 2, cv2.LINE_4)
    cv2.imshow('Drowsiness Detection Tutorial', frame)
    
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()